In [1]:
import d
import random
from nnsight import LanguageModel

random.seed(10)

# Loading raw datasets

In [2]:
variable = "0_forward_belief"

with open(f"data/bigtom/{variable}_true_belief/stories.csv", "r") as f:
    reader = csv.reader(f, delimiter=";")
    tb_data = list(reader)

with open(f"data/bigtom/{variable}_false_belief/stories.csv", "r") as f:
    reader = csv.reader(f, delimiter=";")
    fb_data = list(reader)

with open(f"data/bigtom/{variable}_true_control/stories.csv", "r") as f:
    reader = csv.reader(f, delimiter=";")
    tb_control = list(reader)

with open(f"data/bigtom/{variable}_false_control/stories.csv", "r") as f:
    reader = csv.reader(f, delimiter=";")
    fb_control = list(reader)

# Loading model

In [11]:
model = LanguageModel("meta-llama/Meta-Llama-3-70B")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
priming_examples = """Rewrite the given original question to query about the ground truth regarding than anyone's perspective of the ground thruth. Use the following examples to generate an appropriate response.\nOriginal: Does Noor believe the milk pitcher contains oat milk or almond milk?\nResponse: Does the milk pitcher contains oat milk or almond milk?\n\nOriginal: Does Azola believe the package contains the book she ordered or a hand-knit scarf?\nResponse: Does the package contains the book she ordered or a hand-knit scarf?\n\nOrigial: Does Kavi believe there is a full bottle of hot sauce available or that the hot sauce has been spilled?\nResponse: Is there a full bottle of hot sauce available or that the hot sauce has been spilled?"""

In [12]:
story, question, correct_ans, wrong_ans = tb_data[random.randint(0, len(tb_data) - 1)]
prompt = f"{priming_examples}\n\nOriginal: {question}\nResponse:"

with model.generate("The Eiffel Tower is in the city of", remote=True):
    output = model.generator.output.save()
print("All prediction: ", model.tokenizer.batch_decode(output))

Setting `pad_token_id` to `eos_token_id`:FakeTensor(..., device='meta', size=(), dtype=torch.int64) for open-end generation.
`eos_token_id` should consist of positive integers, but is FakeTensor(..., device='meta', size=(1,), dtype=torch.int64). Your generation will not stop until the maximum length is reached. Depending on other flags, it may even crash.


GuardOnDataDependentSymNode: Could not guard on data-dependent expression u0 (unhinted: u0).  (Size-like symbols: none)

Potential framework code culprit (scroll up for full backtrace):
  File "/home/local_nikhil/.conda/envs/mind/lib/python3.10/site-packages/transformers/models/llama/modeling_llama.py", line 1250, in prepare_inputs_for_generation
    input_ids = input_ids[:, past_length:]

For more information, run with TORCH_LOGS="dynamic"
For extended logs when we create symbols, also add TORCHDYNAMO_EXTENDED_DEBUG_CREATE_SYMBOL="u0"
If you suspect the guard was triggered from C++, add TORCHDYNAMO_EXTENDED_DEBUG_CPP=1
For more debugging help, see https://docs.google.com/document/d/1HSuTTVvYH1pTew89Rtpeu84Ht3nQEFTYhAX3Ypa_xJs/edit?usp=sharing


In [106]:
def get_ground_truth_question(question):
    question = question.split(" ")
    new_question = ""
    if "is" in question:
        is_idx = question.index("is")
        del question[is_idx]
        believe_idx = question.index("believe")
        new_question += "Is the " + " ".join(question[believe_idx + 2:])
        return new_question
    else:
        del question[1], question[1]
        new_question = " ".join(question)
    
    return new_question



with open(f"data/bigtom/0_forward_belief_true_belief/stories.csv", "r") as f:
    reader = csv.reader(f, delimiter=";")
    tb_data = list(reader)

for i in range(len(tb_data)):
    story, question, correct_ans, wrong_ans = tb_data[i]
    print(f"Oringal: {question}")
    print(f"Oringal: {correct_ans}")
    print(f"Oringal: {wrong_ans}")
    print(f"New: {get_ground_truth_question(question)}")

Oringal: Does Noor believe the milk pitcher contains oat milk or almond milk?
Oringal: Noor believes the milk pitcher contains almond milk.
Oringal: Noor believes the milk pitcher contains oat milk.
New: Does the milk pitcher contains oat milk or almond milk?
Oringal: Does Zara believe the painting is the work of a famous artist or an emerging artist?
Oringal: Zara believes the painting is the work of an emerging artist.
Oringal: Zara believes the painting is the work of a famous artist.
New: Is the painting the work of a famous artist or an emerging artist?
Oringal: Does Carlos believe the water valve is closed or open?
Oringal: Carlos believes the water valve is open.
Oringal: Carlos believes the water valve is closed.
New: Is the water valve closed or open?
Oringal: Does Kofi believe his fishing net is in good condition or torn apart?
Oringal: Kofi believes his fishing net is torn apart.
Oringal: Kofi believes his fishing net is in good condition.
New: Is the fishing net in good con

## Using GPT-4

In [80]:
def query_gpt4_chat(messages, model="gpt-4o", temperature=0.2, max_tokens=4096):
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "OpenAI-Organization": api_org
    }

    payload = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload))

    if response.status_code == 200:
        return response.json().get("choices", [{}])[0].get("message", {}).get("content", "")
    else:
        raise Exception(f"Error querying GPT-4 Chat API: {response.text}")

    

In [108]:
import openai
import requests
import json

stories = []
world_question = []
world_correct = []
world_worng  = []

for i in range(len(tb_data)):
    story, question, correct_ans, wrong_ans = tb_data[i]
    stories.append(story)
    print(question, correct_ans, wrong_ans)
    messages = []
    messages.append({"role" : "system", "content" : "I'm doing a research about theory of mind. I have a multiple-choice question phrased to test agent perspective about an occasion that happend or not and I want to rephrase the question to ask about the actualy world state. Here are a few example:\n[AGENT-PERCPECTIVE QUESTION]: Does Noor believe the milk pitcher contains oat milk or almond milk? \n[WORLD-STATE QUESTIONS]: Does the milk pitcher contain oat milk or almond milk?\n\nI'm going to give you an agent-percpective quesion, please return the corresponding world state quesion. Use the following format:\n[WORLD-STATE QUESTIONS]:<the rephrased question>. Do not return anything other than that."})
    messages.append({"role" : "user", "content" : f"[AGENT-PERCPECTIVE QUESTION]: {question}"})
    curr_world_question = query_gpt4_chat(messages)
    world_question.append(curr_world_question.split("[WORLD-STATE QUESTIONS]: ")[-1])
    messages.append({"role" : "assistant", "content" : curr_world_question})
    messages.append({"role" : "user", "content" : f"Thank you! I now want to generate two optional answers to the world-state question that follows the two possible answer in the agent percpective case. For example, the answer in the agent for the example provided before the given options are: (a) Noor believes the milk pitcher contains almond milk. (b) Noor believes the milk pitcher contains oat milk. The corresponding world-state options should be: (a) The milk pitcher contains almond milk. (b) The milk pitcher contains oat milk. The option for the agent percpective question provided by the user are: (a) {correct_ans} (b) {wrong_ans} What are the world-state optional answers corresponding for the agent perspective question provided by the user? Please keep the order of the option as in the agent-pespective case."})
    world_options = query_gpt4_chat(messages)
    world_worng.append(world_options.split("(b) ")[-1])
    world_correct.append(world_options.split("(a) ")[-1].split("(b) ")[0])
    print(world_question[i], world_correct[i],world_worng[i])

    

Does Noor believe the milk pitcher contains oat milk or almond milk? Noor believes the milk pitcher contains almond milk. Noor believes the milk pitcher contains oat milk.
Does the milk pitcher contain oat milk or almond milk? The milk pitcher contains almond milk.
 The milk pitcher contains oat milk.
Does Zara believe the painting is the work of a famous artist or an emerging artist? Zara believes the painting is the work of an emerging artist. Zara believes the painting is the work of a famous artist.
Is the painting the work of a famous artist or an emerging artist? The painting is the work of an emerging artist.
 The painting is the work of a famous artist.
Does Carlos believe the water valve is closed or open? Carlos believes the water valve is open. Carlos believes the water valve is closed.
Is the water valve closed or open? The water valve is open.
 The water valve is closed.
Does Kofi believe his fishing net is in good condition or torn apart? Kofi believes his fishing net is 

Is the coral healthy and thriving or stressed and bleached? The coral is stressed and bleached.
 The coral is healthy and thriving.
Does Heiwa believe the sea is calm and full of fish or that the fish have moved away? Heiwa believes the fish have moved away. Heiwa believes the sea is calm and full of fish.
Is the sea calm and full of fish or have the fish moved away? The fish have moved away.
 The sea is calm and full of fish.
Does Amina believe that her mixed shade of blue paint is still fresh or spilled and dried? Amina believes the shade of blue paint has spilled and dried. Amina believes the shade of blue paint is still fresh and ready to use.
Is the mixed shade of blue paint still fresh or spilled and dried? The shade of blue paint has spilled and dried.
 The shade of blue paint is still fresh and ready to use.
Does Fatima believe there is a hidden chamber based on the hieroglyphs, or does she now know about the concealed door? Fatima knows about the concealed door. Fatima believe

Is the fishing net in perfect condition or torn? The fishing net is torn.
 The fishing net is in perfect condition.
Does Kofi believe the weather is perfect for sailing or is there a storm approaching? Kofi believes there is a storm approaching. Kofi believes the weather is perfect for sailing.
Is the weather perfect for sailing or is there a storm approaching? There is a storm approaching.  
 The weather is perfect for sailing.
Does Lila believe the oven is hot and ready for baking or cool and unsuitable for baking? Lila believes the oven is cool and unsuitable for baking. Lila believes the oven is hot and ready for baking.
Is the oven hot and ready for baking or cool and unsuitable for baking? The oven is cool and unsuitable for baking.
 The oven is hot and ready for baking.
Does Odette believe her accordion is in working condition or has loosened straps? Odette believes her accordion has loosened straps. Odette believes her accordion is in working condition.
Is Odette's accordion in

Is the weather perfect for fishing or is a storm approaching? A storm is approaching.
 The weather is perfect for a day of fishing.
Does Mei believe the rug is in good condition or damp and soaked? Mei believes the rug is damp and soaked. Mei believes the rug is in good condition.
Is the rug in good condition or damp and soaked? The rug is damp and soaked.
 The rug is in good condition.
Does Olabisi believe the bees are calm or agitated and aggressive? Olabisi believes the bees are agitated and aggressive. Olabisi believes the bees are calm.
Are the bees calm or agitated and aggressive? The bees are agitated and aggressive.
 The bees are calm.
Does Kalyani believe the temple floor is clean and perfect for dancing or slippery due to the oil spill? Kalyani believes the floor is slippery due to the oil spill. Kalyani believes the temple floor is clean and perfect for dancing.
Is the temple floor clean and perfect for dancing or slippery due to the oil spill? The floor is slippery due to t

Is the cake mold clean and ready for use or severely dented and unsuitable for use? The cake mold is severely dented and unsuitable for use.
 The cake mold is clean and ready for use.
Does Mei believe the wok is clean and in excellent condition or filled with spilled chili oil? Mei believes the wok is filled with spilled chili oil. Mei believes the wok is clean and in excellent condition.
Is the wok clean and in excellent condition or filled with spilled chili oil? The wok is filled with spilled chili oil.
 The wok is clean and in excellent condition.
Does Jia believe the wooden mold is clean and ready for use or covered in dust? Jia believes the wooden mold is covered in dust. Jia believes the wooden mold is clean and ready for use.
Is the wooden mold clean and ready for use or covered in dust? The wooden mold is covered in dust.
 The wooden mold is clean and ready for use.
Does Enrique believe the seashell is intact or cracked? Enrique believes the seashell is cracked. Enrique believ

Are the flowers in full bloom or have they wilted due to the rainstorm? The flowers have wilted due to the rainstorm.
 The flowers are in full bloom.
Does Aisha believe the water is calm and clear or murky and turbulent? Aisha believes the water is murky and turbulent. Aisha believes the water is calm and clear.
Is the water calm and clear or murky and turbulent? The water is murky and turbulent.
 The water is calm and clear.
Does Janelle believe the coral is healthy or bleached? Janelle believes the coral is bleached. Janelle believes the coral is healthy.
Is the coral healthy or bleached? The coral is bleached.
 The coral is healthy.
Does Naveen believe the firewood is dry and ready to burn or wet and difficult to ignite? Naveen believes the firewood is wet and difficult to ignite. Naveen believes the firewood is dry and ready to burn.
Is the firewood dry and ready to burn or wet and difficult to ignite? The firewood is wet and difficult to ignite.
 The firewood is dry and ready to b

Is the water clear and unpolluted or murky and polluted? The water is murky and polluted.
 The water is clear and unpolluted.
Does Alfonso believe the book is in pristine condition or stained with coffee? Alfonso believes the book is stained with coffee. Alfonso believes the book is in pristine condition.
Is the book in pristine condition or stained with coffee? The book is stained with coffee.
 The book is in pristine condition.
Does Yael believe the book with the intriguing cover has a thrilling plot or a bland storyline? Yael believes the book with the intriguing cover has a thrilling plot. Yael believes the book with the intriguing cover has a bland storyline.
Does the book with the intriguing cover have a thrilling plot or a bland storyline? The book with the intriguing cover has a thrilling plot.  
 The book with the intriguing cover has a bland storyline.
Does Heloise believe the violin's strings are perfectly tuned or detuned? Heloise believes the violin's strings are detuned. 

Are the paint colors bright and lively or dull and muted? The paint colors are now dull and muted.
 The paint colors are still bright and lively.
Does Alejandro believe the yeast is in perfect condition or over-fermented? Alejandro believes the yeast is over-fermented. Alejandro believes the yeast is in perfect condition for baking.
Is the yeast in perfect condition or over-fermented? The yeast is over-fermented.
 The yeast is in perfect condition for baking.
Does Devika believe her oven is at the optimal temperature or has lost heat due to the power outage? Devika believes her oven has lost heat due to the power outage. Devika believes her oven is at the optimal temperature.
Is Devika's oven at the optimal temperature or has it lost heat due to the power outage? Devika's oven has lost heat due to the power outage.
 Devika's oven is at the optimal temperature.
Does Uma believe the vase has a smooth, even texture or has become misshapen? Uma believes the vase has become misshapen. Uma b

Is the central fountain in pristine condition or in a state of disrepair? The central fountain is in a state of disrepair.
 The central fountain is in pristine condition.
Does Percival believe the app's interface is intuitive and user-friendly or corrupted and difficult to use? Percival believes the app's interface is corrupted and difficult to use. Percival believes the app's interface is intuitive and user-friendly.
Is the app's interface intuitive and user-friendly or corrupted and difficult to use? The app's interface is corrupted and difficult to use.
 The app's interface is intuitive and user-friendly.
Does Tanya believe the app's code is well-structured and free of errors or corrupted due to a server malfunction? Tanya believes the app's code is corrupted due to a server malfunction. Tanya believes the app's code is well-structured and free of errors.
Is the app's code well-structured and free of errors or corrupted due to a server malfunction? The app's code is corrupted due to

In [110]:
# Specify the output CSV file
output_file = './data/bigtom_worldstate.csv'
rows = zip(stories, world_question, world_correct, world_worng)


# Write the list to the CSV file with each item in a separate column
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for row in rows:
            writer.writerow(row)
    

In [114]:
stories

["Noor is working as a barista at a busy coffee shop. Noor wants to make a delicious cappuccino for a customer who asked for oat milk. Noor grabs a milk pitcher and fills it with oat milk. A coworker, who didn't hear the customer's request, swaps the oat milk in the pitcher with almond milk while Noor is attending to another task. Noor sees her coworker swapping the milk.",
 'Zara is attending an art exhibition at a local gallery in her town. Zara wants to learn more about a particular abstract painting she admires. The label next to the artwork states that the painting is the work of a famous artist, and Zara reads the label carefully. A gallery staff member, realizing there was a mix-up, changes the label with the correct information attributing the painting to an emerging artist instead. Zara reads the updated label carefully.',
 'Carlos is a farmer in a small village in Mexico, tending to his cornfield. He wants to irrigate his crops by opening a water valve connected to a nearby r

In [120]:
new_stories  = [i.replace('"',' ') for i in stories]

In [128]:
print(world_worng[1])

The painting is the work of a famous artist.
